In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [2]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import callbacks,optimizers,utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os
# import tensorflow as tf

In [3]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.callbacks import LearningRateScheduler
# from tensorflow.keras.optimizers import Adagrad
# from tensorflow.keras.utils import np_utils
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# import matplotlib.pyplot as plt
# import numpy as np
# import os

In [4]:
# determine the total number of image paths in training, validation,
# and testing directories
from imutils import paths
# initialize our number of epochs, initial learning rate, and batch
# size
NUM_EPOCHS = 40
INIT_LR = 1e-2
BS = 32

os.chdir("D:/Dataset/breast-cancer-classification")
trainPaths = list(paths.list_images("datasets_new/training"))
#got all train images
totalTrain = len(trainPaths)
totalVal = len(list(paths.list_images("datasets_new/validation")))
totalTest = len(list(paths.list_images("datasets_new/testing")))

In [5]:

# account for skew in the labeled data
trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels = utils.to_categorical(trainLabels)
classTotals = trainLabels.sum(axis=0)
classWeight = classTotals.max() / classTotals

In [6]:

# initialize the training data augmentation object
trainAug = image.ImageDataGenerator(
rescale=1 / 255.0,
rotation_range=20,
zoom_range=0.05,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.05,
horizontal_flip=True,
vertical_flip=True,
fill_mode="nearest")
 
# initialize the validation (and testing) data augmentation object
valAug = image.ImageDataGenerator(rescale=1 / 255.0)

In [7]:
# initialize the training generator
trainGen = trainAug.flow_from_directory("D:/Dataset/breast-cancer-classification/datasets_new/testing"
,
class_mode="categorical",
target_size=(48, 48),
color_mode="rgb",
shuffle=True,
batch_size=BS)
print("1")
# initialize the validation generator
valGen = valAug.flow_from_directory(
"D:/Dataset/breast-cancer-classification/datasets_new/validation",
class_mode="categorical",
target_size=(48, 48),
color_mode="rgb",
shuffle=False,
batch_size=BS)
print("2")
# initialize the testing generator
testGen = valAug.flow_from_directory(
"D:/Dataset/breast-cancer-classification/datasets_new/testing",
class_mode="categorical",
target_size=(48, 48),
color_mode="rgb",
shuffle=False,
batch_size=BS)

Found 55505 images belonging to 2 classes.
1
Found 22201 images belonging to 2 classes.
2
Found 55505 images belonging to 2 classes.


# MODEL

In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow.keras.backend as K

In [9]:

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers.normalization import BatchNormalization
# from tensorflow.keras.layers.convolutional import SeparableConv2D
# from tensorflow.keras.layers.convolutional import MaxPooling2D
# from tensorflow.keras.layers.core import Activation
# from tensorflow.keras.layers.core import Flatten
# from tensorflow.keras.layers.core import Dropout
# from tensorflow.keras.layers.core import Dense
# from tensorflow.keras import backend as K

In [10]:
inputShape = (48, 48, 3)
chanDim = -1

In [11]:
model = models.Sequential()
model.add(layers.SeparableConv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# (CONV => RELU => POOL) * 2
model.add(layers.SeparableConv2D(64, (3, 3), padding="same"))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.SeparableConv2D(64, (3, 3), padding="same"))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

In [13]:
# (CONV => RELU => POOL) * 3
model.add(layers.SeparableConv2D(128, (3, 3), padding="same"))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.SeparableConv2D(128, (3, 3), padding="same"))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.SeparableConv2D(128, (3, 3), padding="same"))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=chanDim))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

# CLASSIFIER

In [14]:
# first (and only) set of FC => RELU layers
classes = 2
model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
 
# softmax classifier
model.add(layers.Dense(classes))
model.add(layers.Activation("softmax"))
 


# TRAINING

In [15]:
print("[INFO] TRAINING network...")
model.build()
opt = optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
model.compile(
loss = "binary_crossentropy" , optimizer = opt,
metrics = ["accuracy"])

# with tf.device('/cpu:0'):
model.fit_generator(
trainGen,
steps_per_epoch = totalTrain//BS,
validation_data = valGen,
validation_steps = totalVal//BS,

epochs= NUM_EPOCHS
)

[INFO] TRAINING network...
Instructions for updating:
Use tf.cast instead.
Epoch 1/40
1735/1735 [==============================] - 1730s 997ms/step - loss: 0.4218 - acc: 0.8255 - val_loss: 0.4554 - val_acc: 0.8161
Epoch 2/40
1735/1735 [==============================] - 143s 82ms/step - loss: 0.3597 - acc: 0.8465 - val_loss: 0.4255 - val_acc: 0.8263
Epoch 3/40
1735/1735 [==============================] - 138s 80ms/step - loss: 0.3478 - acc: 0.8524 - val_loss: 0.3936 - val_acc: 0.8440
Epoch 4/40
1735/1735 [==============================] - 136s 78ms/step - loss: 0.3417 - acc: 0.8557 - val_loss: 0.3699 - val_acc: 0.8515
Epoch 5/40
1735/1735 [==============================] - 134s 77ms/step - loss: 0.3388 - acc: 0.8563 - val_loss: 0.3782 - val_acc: 0.8495
Epoch 6/40
1735/1735 [==============================] - 139s 80ms/step - loss: 0.3376 - acc: 0.8569 - val_loss: 0.3778 - val_acc: 0.8526
Epoch 7/40
1735/1735 [==============================] - 141s 81ms/step - loss: 0.3355 - acc: 0.8562 -

1735/1735 [==============================] - 139s 80ms/step - loss: 0.3236 - acc: 0.8631 - val_loss: 0.3900 - val_acc: 0.8538
Epoch 37/40
1735/1735 [==============================] - 138s 79ms/step - loss: 0.3241 - acc: 0.8633 - val_loss: 0.3859 - val_acc: 0.8543
Epoch 38/40
1735/1735 [==============================] - 138s 80ms/step - loss: 0.3247 - acc: 0.8624 - val_loss: 0.3882 - val_acc: 0.8547
Epoch 39/40
1735/1735 [==============================] - 147s 85ms/step - loss: 0.3247 - acc: 0.8622 - val_loss: 0.3883 - val_acc: 0.8543
Epoch 40/40
1735/1735 [==============================] - 152s 88ms/step - loss: 0.3240 - acc: 0.8629 - val_loss: 0.3896 - val_acc: 0.8544


# evaluation

In [16]:
print("[INFO] evaluating network...")


[INFO] evaluating network...


In [22]:
predIdxs = model.predict_generator(testGen,
steps=(totalTest // BS) + 1)

In [23]:
predIdxs =  np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs,
target_names=testGen.class_indices.keys()))

              precision    recall  f1-score   support

           0       0.91      0.88      0.89     39793
           1       0.72      0.77      0.75     15712

    accuracy                           0.85     55505
   macro avg       0.81      0.83      0.82     55505
weighted avg       0.85      0.85      0.85     55505



In [25]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [1]:
json_file = open('model.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'model.h5'